In [0]:
import tensorflow as tf
sess = tf.InteractiveSession() 

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [0]:
width = 28
height = 28
flat = width * height
class_output = 10

In [0]:
x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [0]:
x_image = tf.reshape(x, [-1,28,28,1])  
x_image

<h2>Layer 1

In [0]:
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

In [0]:
convolve1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1] , padding='SAME') + b_conv1

In [0]:
h_conv1 = tf.nn.relu(convolve1)

In [0]:
conv1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
conv1

<h2>Layer 2

In [0]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

In [0]:
convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2

In [0]:
h_conv2 = tf.nn.relu(convolve2)

In [0]:
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
conv2

<h2>Layer 3

In [0]:
layer2_matrix = tf.reshape(conv2, [-1, 7*7*64])

In [0]:
W_fc1 = tf.Variable(tf.truncated_normal([7*7*64,1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

In [0]:
fc1 = tf.matmul(layer2_matrix, W_fc1) + b_fc1

In [0]:
h_fc1 = tf.nn.relu(fc1)
h_fc1

In [0]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)

<h2>Output Layer

In [0]:
W_fc2 = tf.Variable(tf.truncated_normal([1024,10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

In [0]:
fc = tf.matmul(layer_drop, W_fc2) + b_fc2

In [0]:
y_CNN = tf.nn.softmax(fc)
y_CNN

<h2>Train & Build

In [0]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))

In [0]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [0]:
correct = tf.equal(tf.argmax(y_CNN,1), tf.argmax(y_,1))

In [0]:
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
sess.run(tf.global_variables_initializer())

In [0]:
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

<h2>Test

In [0]:
# evaluate in batches to avoid out-of-memory issues
n_batches = mnist.test.images.shape[0] // 50
cumulative_accuracy = 0.0
for index in range(n_batches):
    batch = mnist.test.next_batch(50)
    cumulative_accuracy += accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
print("test accuracy {}".format(cumulative_accuracy / n_batches))

In [0]:
sess.close()